# Notebook: Synthetic video and Blending

In [1]:
%load_ext autoreload
%autoreload 2
import cv2
from PIL import Image
import os
from glob import glob
import numpy as np
from tqdm import tqdm
import skvideo.io
from matplotlib import pyplot as plt
import sys
from utils import resize_aspect_ratio

In [2]:
np.set_printoptions(threshold=np.inf)
np.errstate(invalid='ignore', divide='ignore')

In [3]:
_NYU_PATH = 'D:\\data\\dehaze\\nyu\\bedroom_0131'
_REVIDE_PATH = 'D:\\data\\dehaze\\REVIDE_inside\\Train\\gt\\E001_2'
_RESULT_PATH = '../result'
_VIDEO_PATH = 'D:\\data\\dehaze'
_FOG = '../data/fog_square.png'

In [4]:
index_file = open(os.path.join(_NYU_PATH, 'index.txt'), 'r')
index_lines = index_file.readlines()
index_nyu = []
origin_frame = []
depth_frame = []
clear_line = None
depth_line = None
for line in tqdm(index_lines):
    if line.startswith('r'):
        clear_line = os.path.join(_NYU_PATH, line.replace('\n',''))
        origin_frame.append(clear_line)
    elif line.startswith('d'):
        depth_line = os.path.join(_NYU_PATH, line.replace('\n',''))
        depth_frame.append(depth_line)

100%|██████████████████████████████████████████████████████| 4318/4318 [00:00<00:00, 240527.57it/s]


In [5]:
def synthetic_video(vid_name, frames, vid_fps = 60, _type = 'original', resize = True):
    if resize:
        vid_size = sorted(resize_aspect_ratio(frames[0]).shape[:2], reverse=True)
    else:
        vid_size = sorted(frames[0].shape[:2], reverse=True)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_fold = os.path.join(_VIDEO_PATH, _NYU_PATH.split('\\')[-1])
    if not os.path.isdir(out_fold):
        os.mkdir(out_fold)
    out_name = os.path.join(out_fold, _type + '.avi')
    out = cv2.VideoWriter(out_name, fourcc, vid_fps, vid_size)
    for img in tqdm(frames):
        if resize:
            img = resize_aspect_ratio(img)
        out.write(img)
    out.release()
    print('Done synthetic video from images')

In [6]:
def blending(clear, fog):
    """
    Alpha blending
    param:
        clear: is original image path
        fog: is fog image path
    return:
        hazy: is hazy-image
    """
    clear = cv2.imread(clear)
    fog = cv2.imread(fog)
    fog = cv2.resize(fog, clear.shape[1::-1])
    hazy = cv2.addWeighted(clear, 0.5, fog, 0.5, 0)
    return hazy

In [7]:
vid_out_name = _NYU_PATH.split('\\')[-1]+'.avi'
frame_arr = []
for f in tqdm(origin_frame[:200]):
    frame_arr.append(cv2.imread(f))
synthetic_video(vid_out_name, frame_arr, vid_fps = 30, _type = 'original', resize = False)

100%|███████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 257.81it/s]

Done synthetic video from images


In [8]:
vid_out_name = _NYU_PATH.split('\\')[-1]+'.avi'
frame_arr = []
for f in tqdm(origin_frame[:200]):
    h = blending(f,_FOG)
    frame_arr.append(h)
synthetic_video(vid_out_name, frame_arr, vid_fps = 30, _type = 'hazy', resize = False)

100%|███████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 168.26it/s]

Done synthetic video from images


In [9]:
list_imgs = sorted(glob(os.path.join(_REVIDE_PATH, '*.jpg')))
list_hazy = sorted(glob(os.path.join(_REVIDE_PATH.replace('gt','hazy'), '*.jpg')))

In [10]:
len(list_imgs), len(list_hazy)

(66, 66)

In [11]:
# vid_out_name = _REVIDE_PATH.split('\\')[-1]+'.avi'
# frame_arr = []
# for f in tqdm(list_imgs):
#     frame_arr.append(cv2.imread(f))
# synthetic_video(vid_out_name, frame_arr, vid_fps = 60, _type = 'original', resize = True)